In [3]:
import numpy as np
import os
import json
from pprint import pprint

In [4]:
import gensim
from gensim import corpora

unable to import 'smart_open.gcs', disabling that module


## read few files from the dataset

In [5]:
comm_use_subset_files =os.listdir('2020-03-13/comm_use_subset/comm_use_subset')

In [6]:
#all the json files in a directory
#comm_use_subset_files =os.listdir('2020-03-13/comm_use_subset/comm_use_subset')

all_files = []

#open that file as jason and then load
for filename in comm_use_subset_files:
    with open(f'2020-03-13/comm_use_subset/comm_use_subset/{filename}') as json_file:
        file = json.load(json_file)
        all_files.append(file)
    
print(len(all_files))   

9000


## get the abstract for files

In [8]:
#now try with few files only
n_files = 3

all_abstract = []

for f in range(1,n_files+1):
    #print(f)
    file = all_files[f]
    if (file['abstract'] == []):
        pass
    else:
        abstract = file['abstract'][0]['text']
    #pprint(abstract)
    all_abstract.append(abstract)
    
print(all_abstract[:2])

['Dendritic cells (DCs) are specialized antigen-presenting cells (APCs) that have a key role in immune responses because they bridge the innate and adaptive arms of the immune system. They mature upon recognition of pathogens and upregulate MHC molecules and costimulatory receptors to activate antigen-specific CD4 + and CD8 + T cells. It is now well established that DCs are not a homogeneous population but are composed of different subsets with specialized functions in immune responses to specific pathogens. Upon viral infections, plasmacytoid DCs (pDCs) rapidly produce large amounts of IFN-α, which has potent antiviral functions and activates several other immune cells. However, pDCs are not particularly potent APCs and induce the tolerogenic cytokine IL-10 in CD4 + T cells. In contrast, myeloid DCs (mDCs) are very potent APCs and possess the unique capacity to prime naive T cells and consequently to initiate a primary adaptive immune response. Different subsets of mDCs with specializ

## get the titles for files

In [9]:
#now try with few files only
n_files = 100

all_title = []

for f in range(1,n_files+1):
    #print(f)
    file = all_files[f]
    title = file["metadata"]["title"]
    #pprint(file["metadata"])
    all_title.append(title)
    
print(all_title[:3])

['immunity to pathogens taught by specialized human dendritic cell subsets', 'Public Health Responses to and Challenges for the Control of Dengue Transmission in High-Income Countries: Four Case Studies', 'a section of the journal Frontiers in Pediatrics A Review of Pediatric Critical Care in Resource-Limited Settings: A Look at Past, Present, and Future Directions']


## read each line from a file and create a Bag of Words Corpus in gensim

1. tokenize
2. create a dictionary
3. create a doc2bow corpus

In [10]:
from gensim.utils import simple_preprocess

# Tokenize the docs
tokenized_list = [simple_preprocess(line) for line in all_abstract]
#pprint(tokenized_list)

# Create the Corpus, creating a empty corpus ??
#mydict = corpora.Dictionary(tokenized_list) #this will also works
mydict = corpora.Dictionary()
#print(mydict)

mycorpus = [mydict.doc2bow(word, allow_update=True) for word in tokenized_list]
#pprint(mycorpus)

# Computing Pairwise Similarities - 

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

#documents = [open(f) for f in text_files]
tfidf = TfidfVectorizer().fit_transform(all_title)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T

In [13]:
#fill the diagonal elements of the matrix
n, _ = pairwise_similarity.shape  
pairwise_similarity[np.arange(n), np.arange(n)] = -1.0

/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [14]:
pairwise_similarity

<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 7858 stored elements in Compressed Sparse Row format>

In [15]:
#convert the sparse array to a NumPy array via .toarray() or .A
arr = pairwise_similarity.A

In [16]:
arr

array([[-1.        ,  0.0446016 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.0446016 , -1.        ,  0.04368901, ...,  0.01571336,
         0.05185502,  0.03530109],
       [ 0.        ,  0.04368901, -1.        , ...,  0.02660058,
         0.03238621,  0.04301803],
       ...,
       [ 0.        ,  0.01571336,  0.02660058, ..., -1.        ,
         0.05268162,  0.01612003],
       [ 0.        ,  0.05185502,  0.03238621, ...,  0.05268162,
        -1.        ,  0.01297453],
       [ 0.        ,  0.03530109,  0.04301803, ...,  0.01612003,
         0.01297453, -1.        ]])

In [18]:
input_doc = all_title[0]
input_idx = all_title.index(input_doc) + 1 #since all_abstract counting starts at 1 -->range(1,nfiles)
input_idx

1

In [20]:
#query
input_doc

'immunity to pathogens taught by specialized human dendritic cell subsets'

In [22]:
#query
pairwise_similarity[input_idx].argmax() 

46

In [23]:
all_title[pairwise_similarity[input_idx].argmax() ]

'Validation of three geolocation strategies for health-facility attendees for research and public health surveillance in a rural setting in western Kenya'

### So, the `pairwise_similarity` matrix able to find the correct abstract related to the given title. 

### How to find the top 5 most similar articles from `pairwise_similarity` matrix ??

## Alternative approaches to try :

### 1. Create the TF-IDF model

In [24]:
from gensim import models
import numpy as np

# Create the TF-IDF model
tfidf = models.TfidfModel(mycorpus, smartirs='ntc')
#tfidf[mycorpus]

# # Show the TF-IDF weights
# for doc in tfidf[mycorpus]:
#     print([[mydict[id],np.around(freq, decimals=2)] for id, freq in doc]) 


#### Create bigram and trigram using Phrses model, but how it helps ?

In [25]:
#build bigram model from tokenosed list
bigram = gensim.models.phrases.Phrases(tokenized_list, min_count=3, threshold=10)
#print(bigram[mycorpus[0]])
print(bigram[tokenized_list[0]])

['dendritic', 'cells', 'dcs', 'are', 'specialized', 'antigen', 'presenting', 'cells', 'apcs', 'that', 'have', 'key', 'role', 'in', 'immune', 'responses', 'because', 'they', 'bridge', 'the', 'innate', 'and', 'adaptive', 'arms', 'of', 'the', 'immune', 'system', 'they', 'mature', 'upon', 'recognition', 'of', 'pathogens', 'and', 'upregulate', 'mhc', 'molecules', 'and', 'costimulatory', 'receptors', 'to', 'activate', 'antigen', 'specific', 'cd', 'and', 'cd_cells', 'it', 'is', 'now', 'well', 'established', 'that', 'dcs', 'are', 'not', 'homogeneous', 'population', 'but', 'are', 'composed', 'of', 'different', 'subsets', 'with', 'specialized', 'functions', 'in', 'immune', 'responses', 'to', 'specific', 'pathogens', 'upon', 'viral', 'infections', 'plasmacytoid', 'dcs', 'pdcs', 'rapidly', 'produce', 'large', 'amounts', 'of', 'ifn', 'which', 'has', 'potent', 'antiviral', 'functions', 'and', 'activates', 'several', 'other', 'immune', 'cells', 'however', 'pdcs', 'are', 'not', 'particularly', 'potent

In [26]:
#build trigram model
trigram = gensim.models.phrases.Phrases(bigram[tokenized_list], threshold=10)

# Construct trigram
print(trigram[bigram[tokenized_list[0]]])


['dendritic', 'cells', 'dcs', 'are', 'specialized', 'antigen', 'presenting', 'cells', 'apcs', 'that', 'have', 'key', 'role', 'in', 'immune', 'responses', 'because', 'they', 'bridge', 'the', 'innate', 'and', 'adaptive', 'arms', 'of', 'the', 'immune', 'system', 'they', 'mature', 'upon', 'recognition', 'of', 'pathogens', 'and', 'upregulate', 'mhc', 'molecules', 'and', 'costimulatory', 'receptors', 'to', 'activate', 'antigen', 'specific', 'cd', 'and', 'cd_cells', 'it', 'is', 'now', 'well', 'established', 'that', 'dcs', 'are', 'not', 'homogeneous', 'population', 'but', 'are', 'composed', 'of', 'different', 'subsets', 'with', 'specialized', 'functions', 'in', 'immune', 'responses', 'to', 'specific', 'pathogens', 'upon', 'viral', 'infections', 'plasmacytoid', 'dcs', 'pdcs', 'rapidly', 'produce', 'large', 'amounts', 'of', 'ifn', 'which', 'has', 'potent', 'antiviral', 'functions', 'and', 'activates', 'several', 'other', 'immune', 'cells', 'however', 'pdcs', 'are', 'not', 'particularly', 'potent

### 2. create a LSI topic model using gensim?


In [27]:
from gensim.models import LsiModel

# Build the LSI Model
lsi_model = LsiModel(corpus=mycorpus, id2word=mydict, num_topics=7, decay=0.5)


### 3. train Word2Vec model using gensim

In [28]:
# View Topics
pprint(lsi_model.print_topics(-1))### train Word2Vec model using gensim

from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
#import gensim.downloader as api

# Split the data into 2 parts. Part 2 will be used later to update the model
data_part1 = tokenized_list[:1000]
data_part2 = tokenized_list[1000:]

# Train Word2Vec model. Defaults result vector size = 100
model = Word2Vec(data_part1, min_count = 0, workers=cpu_count())

[(0,
  '0.443*"the" + 0.430*"and" + 0.428*"of" + 0.288*"in" + 0.286*"to" + '
  '0.149*"countries" + 0.134*"are" + 0.129*"dengue" + 0.128*"cells" + '
  '0.114*"cd"'),
 (1,
  '0.296*"countries" + -0.285*"cells" + 0.261*"dengue" + -0.253*"cd" + '
  '-0.218*"are" + -0.190*"immune" + -0.158*"mdcs" + -0.127*"dcs" + '
  '-0.127*"subsets" + -0.127*"pathogens"'),
 (2,
  '-0.255*"settings" + -0.255*"care" + -0.235*"critical" + -0.220*"to" + '
  '-0.192*"limited" + -0.192*"mortality" + -0.192*"resource" + 0.182*"dengue" '
  '+ -0.171*"these" + 0.138*"countries"')]


/anaconda3/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [29]:
# Get the vocabulary of word vector (to find any word)
model.wv.vocab

model.most_similar('antiviral')

{'dendritic': <gensim.models.keyedvectors.Vocab at 0x1bb2ba940>,
 'cells': <gensim.models.keyedvectors.Vocab at 0x1bb368e80>,
 'dcs': <gensim.models.keyedvectors.Vocab at 0x1bb373c88>,
 'are': <gensim.models.keyedvectors.Vocab at 0x1bb373cc0>,
 'specialized': <gensim.models.keyedvectors.Vocab at 0x1bb373a58>,
 'antigen': <gensim.models.keyedvectors.Vocab at 0x1bb373e80>,
 'presenting': <gensim.models.keyedvectors.Vocab at 0x1bb373f28>,
 'apcs': <gensim.models.keyedvectors.Vocab at 0x1bb373f60>,
 'that': <gensim.models.keyedvectors.Vocab at 0x1bb373f98>,
 'have': <gensim.models.keyedvectors.Vocab at 0x1bb373fd0>,
 'key': <gensim.models.keyedvectors.Vocab at 0x1bb373710>,
 'role': <gensim.models.keyedvectors.Vocab at 0x1bb39c048>,
 'in': <gensim.models.keyedvectors.Vocab at 0x1bb39c080>,
 'immune': <gensim.models.keyedvectors.Vocab at 0x1bb39c0b8>,
 'responses': <gensim.models.keyedvectors.Vocab at 0x1bb39c0f0>,
 'because': <gensim.models.keyedvectors.Vocab at 0x1bb39c128>,
 'they': <gen

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[('better', 0.29028254747390747),
 ('urgent', 0.2850519120693207),
 ('where', 0.27586880326271057),
 ('virus', 0.2684147357940674),
 ('levels', 0.25499826669692993),
 ('viral', 0.24620981514453888),
 ('subsets', 0.2262580543756485),
 ('cryptic', 0.2248000204563141),
 ('other', 0.2226489931344986),
 ('ago', 0.21679069101810455)]

In [30]:
# Save and Load Model
model.save('newmodel')
model = Word2Vec.load('newmodel')### train Word2Vec model using gensim### train Word2Vec model using gensim